In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
# Imports and matplotlib configuration
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
%matplotlib notebook
import matplotlib.pylab as plt
from matplotlib import animation
from ipywidgets import interact, Button, Output, Box
from IPython.display import display
from style import *

## Universidad Austral de Chile

# INFO337 - Herramientas estadísticas para la investigación

# Linear regression

### A course of the masters in informatics program

### https://github.com/magister-informatica-uach/INFO337

***
<a id="index"></a>

# Contents
***

1. [Introduction](#section1)
1. [The simplest linear model](#section2)
1. [Ordinary least squares](#section3)
1. [Overfitting and regularization](#section4)
1. [Extra topics](#section5)



### References

1. Bishop, "Pattern recognition and machine learning", **Chapter 3**
1. Hastie, Tibshirani and Friedman, "The elements of statistical learning" 2nd Ed., *Springer*, **Chapter 3**
1. Murphy, "Machine Learning: A Probabilistic Perspective", *MIT Press*, 2012, **Chapter 7**



***
<a id="section1"></a>

# Introduction

***

- Many problems can be posed as "finding a relation" between factors/variables
- This can be interpreted as predicting and/or explaining a variable given others
    - Sales given money spent in advertising
    - Chance to rain in Valdivia given temperature, pressure and humidity
    - Gasoline consumption of a car given acceleration, weight and number of cylinders
    - Chance to get lung cancer given number of smoked cigarettes per day, age and gender
- We can ask
    - Are the variable related?
    - How strong/significant is the relationship?
    - What is the nature of the relationship?
- Answering these helps us understand the underlying processes behind the data

***
## Defining regression

- **Regression** refers to a family of statistical methods to find **relationships** between **variables**
- In general the relation is modeled as a function $g(\cdot)$ that maps two types of variables
    - The input variable $X$ is called **independent variable** or feature
    - The output variable $Y$ is called **dependent variable**, response or target
- The mapping or function $g$ is called **predictor** or **regressor**
$$
g: X \to Y
$$
- The objective is to learn $g$ such that we can predict $Y$ given $X$, *i.e.* $\mathbb{E}[Y|X]$ 

## A context for regression

- **Regression** can be defined from an statistical perspective as a special case of model fitting (parameter estimation)
- In many books **Regression** is defined from a pure-optimization perspective (deterministic)
- **Regression** is considered part of the *supervised learning* paradigm
- The difference between **Regression** and *classification* is the nature of the dependent variable (continuous vs categorical)

***
## Parametric vs non-parametric  regression

Regression methods can be broadly classified as either parametric or non-parametric 

In parametric regression
- We know the model of the regressor
- The model has a finite number of parameters
- The parameters of the model are all we need to do predictions 
- Simpler but with bigger assumptions (inductive bias)
- **In this lecture we will focus on parametric regression**

In nonparametric regression
- There is no functional form for the regressor
- It can have an infinite number of parameters (and a finite number of hyperparameters)
- The regressor is defined from the training data
- More flexible but requires more data to fit it
- Examples: Splines, Support vector regression, Gaussian processes

***

## Parametric models for regression

- Let $X$ be a continuous D-dimensional variable (feature)
- Let $Y$ be a continuous unidimensional variable (target) 
- Let $\{x_i, y_i\}$ with $i=1,\ldots,N$ be a set of $N$ *iid* observations of $X$ and $Y$
- Let $g_\theta$ be a model with a M-dimensional parameter $\theta$ 
- Then we can define parametric regression as finding a value of $\theta$ such that 
$$
y_i \approx g_\theta(x_i),\quad i=1,\ldots, N
$$

### Linear models for regression

- The simplest parametric model is the **linear model**
- A linear model gives rise to **linear regression**
- The linear model is linear on $\theta$ but not necessarily on $X$
- For example a model with unidimensional input
$$
g_\theta \left(x_i \right) = \theta_0 + \theta_1 x_i  + \theta_2 x_i^2,
$$
is a linear model and
$$
g_\theta(x_i) = \theta_0 + \theta_1 \log(x_i),
$$
is also a linear model but
$$
g_\theta(x_i) = \theta_0 + \log(x_i + \theta_1),
$$
is not a linear model

[&larr; Go back to the index](#index)

***
<a id="section2"></a>

# The simplest linear model (two-dimensions)
***

If we consider a one-dimensional variable $x_i \in \mathbb{R}, i=1,\ldots,N$, then the simplest linear model is

$$
g_\theta(x_i) = \theta_0 + \theta_1 x_i
$$

which has $M=2$ parameters. 

This corresponds to a line in $\mathbb{R}^2$ and we recognize

- $\theta_0$ as the intercept
- $\theta_1$ as the slope

***
If we consider a two-dimensional variable $x_i = (x_{i1}, x_{i2}) \in \mathbb{R}^2, i=1,\ldots,N$ then we obtain

$$
g_\theta(x_i) = \theta_0 + \theta_1 x_{i1} + \theta_2 x_{i2}
$$

which has $M=3$ parameters. This corresponds to a plane in $\mathbb{R}^3$

***

The most general form assumes a D-dimensional variable $x_i = (x_{i1}, x_{i2}, \ldots, x_{iD}), i=1,\ldots,N$ 

$$
g_\theta(x_i) = \theta_0 + \sum_{j=1}^D \theta_j x_{ij}
$$

which has $M=D+1$ parameters, which corresponds to an hyperplane in $\mathbb{R}^M$

## Fitting the simplest linear model

- Assuming that we have $\{x_i, y_i\}_{i=1,\ldots,N}$ *iid* observations from unidimensional variables X and Y
- How do we find $\theta_0$ and $\theta_1$ such that $y_i \approx \theta_0 + \theta_1 x_i, \forall i$?
- We can start by writing the squared residual (error) as 
$$
E_i^2 = (y_i - \theta_0 - \theta_1 x_i)^2
$$
- We would like to make the sum of squared errors as small as we possible so we seek
$$
\min_{\theta} L = \sum_{i=1}^N E_i^2 = \sum_{i=1}^N (y_i - \theta_0 - \theta_1 x_i)^2,
$$
where $L$, the sum of squares errors, is a our loss/cost function

***

From here we can do

$$
\begin{align}
\frac{dL}{d\theta_0} &= -2 \sum_{i=1}^N (y_i - \theta_0 - \theta_1 x_i) \nonumber \\
&= -2 \sum_{i=1}^N y_i +  2 N\theta_0 + 2 \theta_1 \sum_{i=1}^N x_i = 0 \nonumber
\end{align}
$$

and

$$
\begin{align}
\frac{dL}{d\theta_1} &= -2 \sum_{i=1}^N (y_i - \theta_0 - \theta_1 x_i) x_i \nonumber \\
&= -2 \sum_{i=1}^N y_i x_i +  2 \theta_0 \sum_{i=1}^N x_i + 2 \theta_1 \sum_{i=1}^N x_i^2 = 0 \nonumber
\end{align}
$$

a system of two equations and two unknowns

$$
\begin{pmatrix} N & \sum_i x_i \\ \sum_i x_i & \sum_i x_i^2\\\end{pmatrix} \begin{pmatrix} \theta_0 \\ \theta_1 \end{pmatrix}  = \begin{pmatrix} \sum_i y_i \\ \sum_i x_i y_i \end{pmatrix} 
$$

whose solution is 

$$
\begin{pmatrix} \hat \theta_0 \\ \hat \theta_1 \end{pmatrix}  = 
\frac{1}{N\sum_i x_i^2 - (\sum_i x_i)^2}\begin{pmatrix} \sum_i x_i^2 & -\sum_i x_i \\ -\sum_i x_i & N\\\end{pmatrix}  
\begin{pmatrix} \sum_i y_i \\ \sum_i x_i y_i \end{pmatrix} 
$$

where we assume that the determinant complies with: $N\sum_i x_i^2 - (\sum_i x_i)^2 \neq 0$

Note that in this case

$$
\hat \theta_1 = \frac{\text{Cov}[x, y]}{\text{Var}[x]}, \hat \theta_0 = \bar y - \hat \theta_1 \bar x
$$

In [ ]:
plt.close('all'); fig = plt.figure(figsize=(10, 7))
theta = np.random.randn(2); display(repr(theta))
x = np.linspace(-5, 5, num=10);
def update(rseed, sigma):
    np.random.seed(rseed); 
    y = theta[0] + theta[1]*x + sigma*np.random.randn(len(x))
    sum_x2 = np.sum(x**2); sum_x = np.sum(x)
    sum_xy = np.sum(x*y); sum_y = np.sum(y)
    theta_hat = np.dot([[sum_x2, -sum_x],[-sum_x, len(x)]], [sum_y, sum_xy])/(len(x)*sum_x2 - sum_x**2)
    display(repr(theta_hat))
    ax = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax.set_ylim([-10, 10])
    ax.scatter(x, y, c='r', s=100, label='data')
    ax.vlines(x, theta_hat[0] + theta_hat[1]*x, y, 'r')
    ax.plot(x, theta[0] + theta[1]*x, 'b--', linewidth=4, alpha=0.75, label='underlying'); 
    ax.plot(x, theta_hat[0] + theta_hat[1]*x, 'k-', linewidth=4, label='model'); plt.legend();
    ax = plt.subplot2grid((3, 1), (2, 0))
    ax.plot(x, np.zeros_like(x), 'k--', alpha=0.5)
    ax.scatter(x, y - theta_hat[0] - theta_hat[1]*x, c='r', s=100);
    ax.set_ylabel('Residuals'); 
    
interact(update, rseed=IntSlider_nice(), sigma=SelectionSlider_nice(options=[0.1, 0.5, 1, 5.]));

## Quality of the linear fit

Can we measure how strong is $y$ related to $\hat y = \hat \theta_0 + \hat \theta_1 x$?

YES: **Coefficient of determination** or $r^2$
$$
r^2 = 1 - \frac{\sum_i (y_i - \hat y_i)^2}{\sum_i (y_i - \bar y_i)^2}
$$
- one minus the Sum of residuals divided by the variance of y
- $r$ is equivalent to Pearson's correlation coefficient
- $r^2 \in [0, 1]$
- If $r^2 = 1$, the data points are fitted perfectly by the model. The regressor accounts for all of the variation in y
- If $r^2 = 0$, the regression line is horizontal. The regressor accounts for none of the variation in y
- If the relation is strong but non-linear it will not be detected by $r^2$

[&larr; Go back to the index](#index)

***
<a id="section3"></a>

# Ordinary Least Squares (OLS)
***

- Now we will generalize the previous solution
- Assuming that we have $\{x_i, y_i\}_{i=1,\ldots,N}$ *iid* observations from unidimensional  Y and **D-dimensional variable** X
- How do we find $\theta$  such that $y_i \approx \theta_0 + \sum_{j=1}^D \theta_j x_{ij}, \forall i$?
- We can write the sum of squared errors (residuals) as
$$
\min_\theta L = \sum_{i=1}^N (y_i - \theta_0 - \sum_{j=1}^D \theta_j x_{ij})^2
$$

- For simplicity consider the matrices
$$
X = \begin{pmatrix} 1 & x_{11} & x_{12} & \ldots & x_{1D} \\ 
1 & x_{21} & x_{22} & \ldots & x_{2D} \\
1 & \vdots & \vdots & \ddots & \vdots \\
1 & x_{N1} & x_{N2} & \ldots & x_{ND} \end{pmatrix},  Y = \begin{pmatrix} y_1 \\ y_2 \\ \vdots \\ y_N \end{pmatrix}, \theta =  \begin{pmatrix} \theta_0 \\ \theta_1 \\ \vdots \\ \theta_D \end{pmatrix}
$$
- We can write the cost function in matrix form as
$$
\min_\theta  L = \| Y - X \theta \|^2 = (Y - X \theta)^T (Y - X \theta)
$$

***

From here we can do

$$
\frac{dL}{d\theta} = -(Y - X \theta)^T X =  -X^T (Y - X \theta) = 0
$$

to obtain the **normal equations**

$$
X^T X \theta  = X^T Y
$$

and if $X^T X$ is invertible then

$$
\hat \theta = (X^T X)^{-1} X^T Y
$$

which  is called the **least squares (LS) estimator** of $\theta$

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
plt.close('all'); fig = plt.figure(figsize=(7, 5))
ax = fig.add_subplot(111, projection='3d')
theta = [4, 3, 2]; display(repr(theta))
x = np.linspace(-5, 5, num=10);
X, Y = np.meshgrid(x, x)
XY1 = np.stack((np.ones_like(X.ravel()), X.ravel(), Y.ravel())).T
def update(rseed, sigma):
    np.random.seed(rseed); 
    Z_clean = theta[0] + theta[1]*X + theta[2]*Y 
    Z = Z_clean + sigma*np.random.randn(len(x), len(x))
    param, MSE, rank, singval = np.linalg.lstsq(XY1, Z.ravel(), rcond=None)
    display(repr(param))
    ax.cla();
    ax.scatter(X, Y, Z, s=100, c='r', label='data')
    ax.plot_wireframe(X, Y, Z_clean, label='underlying')
    ax.plot_wireframe(X, Y, param[0] + param[1]*X + param[2]*Y, color='k', label='model')
    plt.legend()
    
interact(update, rseed=IntSlider_nice(), sigma=SelectionSlider_nice(options=[0.1, 1, 10.]));

For the two-dim linear model we found these normal equations

$$
\begin{pmatrix} N & \sum_i x_i \\ \sum_i x_i & \sum_i x_i^2\\\end{pmatrix} \begin{pmatrix} \theta_0 \\ \theta_1 \end{pmatrix}  = \begin{pmatrix} \sum_i y_i \\ \sum_i x_i y_i \end{pmatrix} 
$$

and they can be written as
$$
\begin{align}
\begin{pmatrix} 1 & 1 & \ldots & 1 \\ x_1 & x_2 & \ldots & x_N \end{pmatrix} 
\begin{pmatrix} 1 & x_1 \\ 1 & x_2 \\ \vdots & \vdots \\ 1 & x_N \end{pmatrix} 
\begin{pmatrix} \theta_0 \\ \theta_1 \end{pmatrix}  &= 
\begin{pmatrix} 1 & 1 & \ldots & 1 \\ x_1 & x_2 & \ldots & x_N \end{pmatrix} 
\begin{pmatrix} y_1 \\ y_2 \\ \vdots \\ y_N \end{pmatrix} \nonumber \\
X^T X \theta &= X^T Y \nonumber
\end{align}
$$
the same form as the general case


***

### Numerical considerations of the LS solution

The LS estimator is

$$
\hat \theta = (X^T X)^{-1} X^T Y
$$

- The LS estimator is a batch solution, *i.e.* it uses all observations at once
    - If N is large it might be impossible to obtain it
    - In that case it might be better to use stochastic gradient descent 
- $X^{\dagger} = (X^T X)^{-1} X^T $ is known as the *Moore-Penrose* pseudo-inverse
    - $X^{\dagger}$ is the inverse for matrices that are not squared
    - If X is squared and invertible then $X^{\dagger} = (X^T X)^{-1} X^T  = X^{-1} (X^T)^{-1} X^T = X^{-1}$
- The LS estimator is valid only if we can invert $X^T X$
    - $X^T X$ is a squared symmetric $M\times M$ matrix
    - If $(X^T X)^{-1}$ exists the LS solution is unique and the problem is *well-posed*
    - If $X^T X$ is positive definite, *i.e.* $z^T X^T X z > 0, \forall z$ then its inverse exist
    - $X^T X$ being positive definite is equivalent to each of the following
        - The columns of $X$ are Linearly Independent (LI)
        - All the eigenvalues of $X^T X$ are positive
        - The Cholesky decomposition of $X^T X$ exists
- **Trick of the trade:** Add a small value to the diagonal of $X^T X$ to make it invertible
    - We will see later that this is equivalent to regularization

## Quality of D-dimensional linear fit

In this case the **Coefficient of determination** or $r^2$ can be written as
$$
\begin{align}
r^2 &= 1 - \frac{\sum_i (y_i - \hat y_i)^2}{\sum_i (y_i - \bar y_i)^2} \nonumber \\
&= 1 - \frac{Y^T(I-X(X^TX)^{-1}X^T)Y}{Y^T (I - \frac{1}{N} \mathbb{1}^T \mathbb{1} ) Y} \nonumber \\
&= 1 - \frac{SS_{res}}{SS_{total}} \nonumber
\end{align}
$$
where $\mathbb{1} = (1, 1, \ldots, 1)$
- It has the same interpretation as explained before
- Review the previous class on ANOVA and reflect on the similarities, How do we get F distributions from $SS_{res}$ and $SS_{total}$?


***

# Statistical view of OLS

- Up to now we have viewed regression from a deterministic (optimization) perspective
- To understand the properties and do inference we seek an statistical interpretation
- Assuming that we have $\{x_i, y_i\}_{i=1,\ldots,N}$ iid observations from unidimensional  Y and **D-dimensional variable** X
- Let's assume that our measurements of $Y$ consists of the **true model** plus **white Gaussian noise**, *i.e.*

$$
y_i = \theta_0 + \sum_{j=1}^D \theta_j x_{ij} + \varepsilon_i, \forall i \quad \text{and} \quad \varepsilon_i \sim \mathcal{N}(0, \sigma^2)
$$

- Then we can write the log likelihood of $\theta$ as 

$$
\begin{align}
\log L(\theta) &= \log \prod_{i=1}^N \mathcal{N}(y_i | (1, x_{i1}, \ldots, x_{iD}) \theta, \sigma^2) \nonumber \\
&= \sum_{i=1}^N \log \mathcal{N}(y_i | (1, x_{i1}, \ldots, x_{iD}) \theta, \sigma^2) \nonumber \\
&= -\frac{N}{2} \log(2\pi \sigma^2) - \frac{1}{2\sigma^2} \sum_{i=1}^N (y_i - (1, x_{i1}, \ldots, x_{iD}) \theta)^2,\nonumber \\
&= -\frac{N}{2} \log(2\pi \sigma^2) - \frac{1}{2\sigma^2} (Y-X\theta)^T (Y - X\theta) \nonumber 
\end{align}
$$


### Maximum Likelihood of Spherical Gaussian and Least Squares

Given that we know $\sigma > 0$ then

$$
\begin{align}
\max_\theta \log L(\theta) &= -\frac{N}{2} \log(2\pi \sigma^2) - \frac{1}{2\sigma^2} (Y-X\theta)^T (Y - X\theta) \nonumber \\
&= - \frac{1}{2\sigma^2} (Y-X\theta)^T (Y - X\theta) \nonumber 
\end{align}
$$
We can change the sign and transform *max* to *min*:

$$
\begin{align}
\min_\theta \log L(\theta) &=  \frac{1}{2\sigma^2} (Y-X\theta)^T (Y - X\theta) \nonumber \\
\implies \hat \theta &= (X^T X)^{-1} X^T Y \nonumber 
\end{align}
$$

> In this case the MLE solution is equivalent to least squares

> When using the sum of squares as cost function we are assuming that the noise is Spherical Gaussian distributed

***

# Statistical properties of OLS

- Let $\varepsilon = (\varepsilon_1, \varepsilon_2, \ldots, \varepsilon_N)$, where $\varepsilon_i \sim \mathcal{N}(0, \sigma^2) \forall i$ and assume that X is not random
- Is the estimator unbiased?
$$
\begin{align}
\mathbb{E}[\hat \theta] &= \mathbb{E}[(X^T X)^{-1} X^T Y] \nonumber \\
&= \mathbb{E}[(X^T X)^{-1} X^T (X \theta + \varepsilon)] \nonumber \\
&= \mathbb{E}[\theta] + (X^T X)^{-1} X^T \mathbb{E}[\varepsilon] \\
& = \mathbb{E}[\theta]
\end{align}
$$
YES! 
- What is the variance of the estimator? 
$$
\begin{align}
\mathbb{E}[(\hat \theta - \mathbb{E}[\hat\theta])(\hat \theta - \mathbb{E}[\hat\theta])^T] &= \mathbb{E}[((X^T X)^{-1} X^T \varepsilon) ((X^T X)^{-1} X^T \varepsilon)^T] \nonumber \\
&= (X^T X)^{-1} X^T  \mathbb{E}[\varepsilon \varepsilon^T] X ((X^T X)^{-1})^T  \nonumber \\
&= (X^T X)^{-1} X^T  \mathbb{E}[(\varepsilon-0) (\varepsilon-0)^T] X (X^T X)^{-1}  \nonumber \\
& =\sigma^2 (X^T X)^{-1}
\end{align}
$$
because the variance of $\varepsilon$ is $I\sigma^2$
- Typically we estimate the variance of the noise using the unbiased estimator
$$
\begin{align}
\hat \sigma^2 &= \frac{1}{N-D-1} \sum_{i=1}^N (y_i - \theta_0 - \sum_{j=1}^D \theta_j x_{ij})^2 \nonumber \\
& = \frac{1}{N-D-1} (Y-X\theta)^T (Y-X\theta)
\end{align}
$$
- (Hastie, 3.2.2) **The Gauss-Markov Theorem:** The least squares estimate of $\theta$ have the smallest variance among all unbiased estimators

***

## Inference with OLS

- Previously we found the expected value and the variance of $\theta$
- From the properties of MLE we know that
$$
    \hat \theta \sim \mathcal{N}(\theta, \sigma^2 (X^T X)^{-1})
$$
and the estimator of the variance will be proportional to
$$
\hat \sigma^2 \sim  \frac{1}{(N-M)}\sigma^2 \chi_{N-M}^2
$$
- With this we have all the ingredients to find confidence intervals and do hypothesis test on $\hat \theta$


### Hypothesis test on the regression coefficients

To assess the significance of our model we might try to reject the following *hypotheses*

- One of the parameters (slopes) is zero

    $\mathcal{H}_0: \theta_i = 0$
    
    $\mathcal{H}_A: \theta_i \neq 0$
    
    
- All parameters are zero 

    $\mathcal{H}_0: \theta_1 = \theta_2 = \ldots = \theta_D = 0$

    $\mathcal{H}_A:$ At least one parameter is not zero


- A subset of the parameters are zero 

    $\mathcal{H}_0: \theta_i = \theta_j =0 $

    $\mathcal{H}_A:$ $\theta_i \neq 0 $ or $\theta_j \neq 0 $


> We can do this using t-test, f-test or ANOVA

### Example

Read more on this at:
- https://onlinecourses.science.psu.edu/stat501/node/295/
- http://reliawiki.org/index.php/Simple_Linear_Regression_Analysis


We can trust the test only if our assumptions are true

- Relation between X and Y is linear
- Errors/noise are *iid* 
- Errors/noise follows a multivariate normal with covariance $I\sigma^2$

### What if the assumptions are incorrect

We have to prove them right: **Residual analysis**

1. Check the residuals for normality, Are there outliers that we should remove?
1. Check for absence of correlation in the residuals
1. Do the errors have different variance ?



For the latter we can use the  **Weighted Least Squares** 


***

## Weighted Least Squares (WLS)
***

- Before we assumed that the noise was homoscedastic (constant variance). We will generalize to the heteroscedastic case
- Assuming that we have $\{x_i, y_i\}_{i=1,\ldots,N}$ *iid* observations from unidimensional  Y and **D-dimensional variable** X
- How do we find $\theta$  such that $y_i \approx \theta_0 + \sum_{j=1}^D \theta_j x_{ij}, \forall i$?
- Let's assume that our measurements of $Y$ consists of the **true model** plus **Gaussian noise** with changing variance, *i.e.*

$$
    y_i = \theta_0 + \sum_{j=1}^D \theta_j x_{ij} + \varepsilon_i, \forall i \quad \text{and} \quad \varepsilon_i \sim \mathcal{N}(0, \sigma_i^2)
$$

- In this case the maximum likelihood solution is 
$$
\hat \theta = (X^T \Sigma^{-1}X)^{-1} X^T \Sigma^{-1} Y
$$
where
$$
\Sigma = \begin{pmatrix} 
\sigma_1^2 & 0 &\ldots & 0 \\
0 & \sigma_2^2 &\ldots & 0 \\
\vdots & \vdots &\ddots & \vdots \\
0 & 0 &\ldots & \sigma_N^2 \\
\end{pmatrix}
$$

- Note that in this case
$$
\hat \theta \sim \mathcal{N}( \theta,  (X^T X)^{-1} X^T  \Sigma X (X^T X)^{-1} )
$$


## Linear models and Basis functions

- Can we estimate a model that is not an hyperplane? YES!

- The most general form of a linear model is
$$
g_\theta(x_i) = \sum_{j=0}^M \theta_j \phi_j(x_i),
$$
where $\phi_j: \mathbb{R}^D \to \mathbb{R}$ is a set of basis functions
- Note that M and D are not related as before

### Example: Polynomials

- Unidimensional variable $x_i \in \mathbb{R}, i=1,\ldots,N$
- Basis function is 
$$
\begin{align}
\phi_0 (x) &= 1 \nonumber \\
\phi_1 (x) &= x \nonumber \\
\phi_2 (x) &= x^2 \nonumber \\
& \vdots \nonumber \\
\phi_M (x) &= x^M \nonumber \\
\end{align}
$$
- This yields a M-degree polynomial model

$$
g_\theta(x_i) = \theta_0 + \theta_1 x_i + \theta_2 x_i^2 + \ldots + \theta_M x_i^M, \forall i
$$

- It is linear in the parameters but non-linear in the input

### Example: Transformations

- Unidimensional variable $x_i \in \mathbb{R}, i=1,\ldots,N$
- Basis function is 

$$
\phi_j(x_i) = \cos(2\pi j x_i/P)
$$

- This yields

$$
g_\theta(x_i) = \theta_0 + \theta_1 \cos(2\pi x_{i}/P) + \theta_2 \cos(4\pi x_{i}/P) + \ldots + \theta_M \cos(2M\pi x_{i}/P), \forall i
$$

- A periodic model that is linear on $\theta$

### Example: Interactions

- Bidimensional variable $x_i = (x_{i1}, x_{i2}), i=1,\ldots,N$
- Basis function is 

$$
\phi_j(x_i) = 
\begin{cases} 
1 & j=0 \\ x_{i1} & j=1 \\ 
x_{i2} & j=2 \\ x_{i1} x_{i2} & j=3 \\ 
x_{i1}^2 & j=4 \\ x_{i2}^2 & j=5 \\
\end{cases}
$$
- A basis with interactions between variables up to second-degree

### Example: The simplest linear model

- D-dimensional variable $x_i = (x_{i1}, x_{i2}, \ldots, x_{iD}), i=1,\ldots,N$
- Basis function is 


$$
\phi_j(x_i) = \begin{cases} 1 & j=0 \\ x_{ij} & j \in [1,D] \end{cases}
$$


- This yields

$$
g_\theta(x_i) = \theta_0 + \theta_1 x_{i1} + \theta_2 x_{i2} + \ldots + \theta_M x_{iD}, \forall i
$$

- Which is the simplest linear model we reviewed before

***
## Practical example: Polynomial regression

- Play with $M$ the polynomials degree and $\sigma$ the variance of the noise
- What happens when $M > N$? Does it depend on $\sigma$?
- What happens to the prediction on the test data point?

In [ ]:
def gen_basis(x, M=1):
    X2 = list()
    for i in range(M):
        X2.append(x**i)
    return np.vstack(X2).T

plt.close('all'); fig = plt.figure(figsize=(10, 7))
x = np.linspace(-5, 6, num=11); 
x2 = np.linspace(-5, 6, num=100);
theta = [10, -2, -0.3, 0.1]
X = gen_basis(x, M=len(theta))
print(repr(theta))

def update(sigma, rseed, M):
    np.random.seed(rseed); Y = np.dot(X, theta) + sigma*np.random.randn(len(x))
    X2 = gen_basis(x, M)
    theta_hat = np.linalg.solve(np.dot(X2[:10, :].T, X2[:10, :]), np.dot(X2[:10, :].T, Y[:10]))
    print(repr(theta_hat))    
    ax = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax.scatter(x[:10], Y[:10], c='r', s=100, label='train data')
    ax.scatter(x[10:], Y[10:], c='g', s=100, label='test data')
    ax.vlines(x[:10], np.dot(X2[:10, :], theta_hat), Y[:10], 'r')  
    ax.vlines(x[10:], np.dot(X2[10:, :], theta_hat), Y[10:], 'g') 
    ax.plot(x, np.dot(X, theta), 'b--', linewidth=4, label='underlying')
    ax.plot(x2, np.dot(gen_basis(x2, M), theta_hat), 'k-', linewidth=4, label='model')
    ax.set_ylim([-5, 15]); plt.legend()
    ax = plt.subplot2grid((3, 1), (2, 0))
    ax.plot(x, np.zeros_like(x), 'k--', alpha=0.5)
    ax.scatter(x, Y - np.dot(X2, theta_hat), c='k', s=100); 
    
interact(update, rseed=IntSlider_nice(), M=SelectionSlider_nice(options=[1, 2, 3, 4, 5, 7, 10]), 
         sigma=SelectionSlider_nice(options=[0.1, 1, 2, 5]));

[&larr; Go back to the index](#index)

***
<a id="section4"></a>

# Overfitting and regularization
***

- In the previous example $M$ represents the complexity of the model
- In general more complex models give more flexibility to fit the data
- But too much complexity causes **overfitting**
    - the model fits the noise
    - we can't extract the underlying behavior 
    - the model **does not generalize** to new data
- Ways to avoid overfitting
    - Using low complexity models
    - Set complexity using cross-validation 
    - Regularization

***

## The Bias-Variance trade-off

- Let's assume that the data is $y = f(x) + \varepsilon$ (true model + Gaussian noise)
- We use a linear model to find $f(x)$ as $\hat y = \sum_j \theta_j \phi_j(x)$
- We can measure the quality of our model with the Mean Square Error (MSE)
$$
\begin{align}
\mathbb{E}[(y - \hat y)^2] &= \mathbb{E}[y^2 -2 y \hat y +\hat y^2] \nonumber \\
&= \mathbb{E}[(f+\varepsilon)^2 -2 (f+\varepsilon) \hat y +\hat y^2] \nonumber \\
&= \mathbb{E}[(f^2 +2 f \varepsilon + \varepsilon^2 -2 (f+\varepsilon) \hat y +\hat y^2] \nonumber \\
&= \mathbb{E}[\varepsilon^2] + f^2  -2 f \mathbb{E}[\hat y]  +\mathbb{E}[\hat y^2]  \pm \mathbb{E}[\hat y]^2  \nonumber \\
&= \mathbb{E}[\varepsilon^2] + (f - \mathbb{E}[\hat y])^2  +\mathbb{E}[(\hat y - \mathbb{E}[\hat y])^2]  \nonumber \\
&= \sigma^2 + (f - \mathbb{E}[\hat y])^2  + \text{Var}[\hat y]  \nonumber 
\end{align}
$$
- The MSE can be decomposed as irreducible error + squared bias of the estimator + variance of the estimator
- The MSE can be small of either the bias or the variance are small
- Typically, the more complex the model the lower the bias and the higher the variance it attains 
- The Gauss-Markov theorem says that OLS has the minimum variance of the unbiased estimator. 
- But unbiased is not necessarily good (overfit). In some cases we may want to trade bias for variance
- This is achieved by penalizing the complexity of the model: **Regularization**


***

# Bayesian Least Squares and Ridge regression


If we assume a Gaussian likelihood and a Gaussian prior we can write the log joint as

$$
\begin{align}
\log p({x}, \theta) &= \log \prod_{i=1}^N \mathcal{N}(y_i | (1, x_{i1}, \ldots, x_{iD}) \theta, \sigma^2) + \log \prod_{j=1}^M \mathcal{N}(\theta_j | 0, \sigma_0^2) \nonumber \\
&= -\frac{N}{2} \log(2\pi \sigma^2) - \frac{1}{2\sigma^2} (Y-X\theta)^T (Y - X\theta) -\frac{M}{2} \log(2\pi \sigma_0^2) - \frac{1}{2\sigma_0^2} \|\theta\|^2 \nonumber 
\end{align}
$$

The MAP estimator of $\theta$ is given by
$$
\begin{align}
\hat \theta &= \text{arg}\max_\theta \log p({x}, \theta) \nonumber  \\
&= \text{arg}\max_\theta  - \frac{1}{2\sigma^2} (Y-X\theta)^T (Y - X\theta) - \frac{1}{2\sigma_0^2} \|\theta\|^2 \nonumber \\
&= \text{arg}\min_\theta  (Y-X\theta)^T (Y - X\theta) + \lambda \|\theta\|^2 \nonumber
\end{align}
$$
where $\lambda = \frac{\sigma^2}{\sigma_0^2}$

The solution is obtained by taking the derivative on $\theta$

$$
\frac{d}{d\theta} (Y-X\theta)^T (Y - X\theta) + \lambda \|\theta\|^2  = -X^T (Y - X\theta) + \lambda \theta = 0
$$

and finally

$$
\hat \theta = (X^T X + \lambda I)^{-1} X^T Y
$$

- This is known as Ridge regression and Tikhonov regularization
- We are forcing the solution to be smooth (have small L2 norm)
- This can help to avoid overfitting with complex models 
- It can also help when $X^T X$ is not invertible
    - Adding a $\lambda$ to the diagonal of $X^T X$ makes the eigenvalues positive (unique solution)
- (There is no free lunch) How to choose $\lambda$?
    - Cross-validation: minimize validation error
    - L-curve: Plot $ \log (Y-X\theta)^T (Y - X\theta)$ vs $ \log \|\theta\|^2$ and find the elbow
- Note that different priors yield different regularization schemes
    - A Laplacian prior yields a L1 norm which forces the solution to be sparse (LASSO)

In [ ]:
plt.close('all'); fig, ax = plt.subplots(figsize=(7, 4))
x = np.linspace(-5, 6, num=50); x_plot = np.linspace(-5, 6, num=200); 
model = np.sin(x)*x + 0.1*x**2
print(repr(theta))

def update(sigma, rseed, lamb, M):
    np.random.seed(rseed); 
    y = model + sigma*np.random.randn(len(x))
    regressor = make_pipeline(PolynomialFeatures(M), Ridge(normalize=True, alpha=lamb))
    regressor.fit(x.reshape(-1, 1), y)
    ax.cla(); ax.plot(x, model, 'b--', linewidth=4, label='underlying')
    ax.plot( x_plot , regressor.predict( x_plot .reshape(-1, 1)), 'k-', linewidth=4, label='model')
    ax.scatter(x, y, c='r', s=30, label='data', zorder=100); plt.legend()

    
interact(update, rseed=IntSlider_nice(), M=SelectionSlider_nice(options=[1, 2, 3, 5, 10, 20]), 
         sigma=SelectionSlider_nice(options=[0.1, 1, 2, 5]),
         lamb=SelectionSlider_nice(options=[0.0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1., 100000.]));

[&larr; Go back to the index](#index)


***
<a id="section5"></a>

# Extra topics
***

Suggestions if you want to go deeper 
- (Hastie 3.4 and 3.8) L1 regularization and Least Absolute Shrinkage and Selection Operator (LASSO)
- Robust regression: Least absolute regression and M-estimators for data with outliers (non-Gaussian)
- (Hastie 6 & Bishop 6) Kernel (non-parametric) regression  
- Some of these topics can be found at [Huijse, Regresión](https://docs.google.com/presentation/d/1UUpK4zSdzRcS79V7_wU9nXe-sR7qYLEWhbmid-Rfp1k/edit#slide=id.g28044c0f85_0_34)


# Independence and correlation

Two variables can have zero correlation but still be dependent

> Linear regression (correlation) is sensitive to linear relationships

More generally, to test indepedence we could use:

$$
p(x,y) = p(x)p(y)
$$

Several methods are based on this
- Shannon's **Mutual Information**
$$
I(X,Y) = \int \int f_{XY}(x,y) \log \frac{f_{XY}(x,y)}{f_{X}(x) f_Y(y)} dx dy
$$
- [Correlation distance](https://arxiv.org/pdf/0803.4101.pdf)
$$
R(X,Y) = \int \int |f_{XY}(x,y)  - f_{X}(x) f_Y(y)| dx dy
$$

Although these require that we estimate the joint and the marginals (KDE, Histogram, Parametric)

For categorical variables we can use the **chi square test**